# ORM, WEB API

## 학습목표 
 1. SQLAlchemy ORM을 이용한 mysql 데이터 다루기
 2. Flask 설치 및 API 실습
 3. cron, tmux의 이해 및 숙지

* ORM
  - Object Relation Mapping
  - DB에서 데이터를 가져오거나 데이터를 저장할 때, tuple이나 dict등의 자료가 아닌 해당 row에 맞는 객체를 이용하는 방법
  - 각 언어와 환경에 맞는 다양한 ORM이 존재.
  - python의 경우 SQLAlchemy가 가장 많이 사용 됨.

In [2]:
#server = 
connection_string = 'mysql+mysqldb://root:pass@{}:3306/test'.format(server)
print connection_string

NameError: name 'server' is not defined

* model 생성

In [14]:
# -*- coding: utf-8 -*-

#import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')

import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, CHAR, Date, String, Time, Index, DateTime, TIMESTAMP, func
from sqlalchemy.dialects.mysql import INTEGER, BIT, TINYINT, TIME, DOUBLE, TEXT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class Student(Base):
    __tablename__ = 'students'

    ID              = Column(Integer, primary_key = True, nullable = False, autoincrement = True)
    Name            = Column(CHAR(35), nullable = False)
    Age             = Column(Integer, nullable = False, default = 25)
    MajorCode       = Column(String(10), nullable = True, default = None)
    
class Major(Base):
    __tablename__ = 'majors'
    
    ID              = Column(Integer, primary_key = True, nullable = False, autoincrement = True)
    Code            = Column(String(10), nullable = False)
    Name            = Column(String(45), nullable = False)
    Description     = Column(String(45), nullable = False, default = None)
    

class Course(Base):
    __tablename__ = 'courses'
    
    Code            = Column(String(20), primary_key = True, nullable = False, unique = True)
    Name            = Column(String(100), nullable = False)
    
class Score(Base):
    __tablename__ = 'scores'
    
    ID              = Column(Integer, primary_key = True, nullable = False, autoincrement = True)
    StudentID       = Column(Integer, nullable = False)
    CourseCode      = Column(String(20), nullable = False)
    Score           = Column(Integer, nullable = False, default = 0)

In [15]:
print 'hello'

hello


* select

In [17]:

engine = create_engine(connection_string, pool_recycle = 3600, encoding='utf-8')
Session = sessionmaker(bind=engine)

session = Session()

#result = session.query(Student)  \  #select
#        .filter(Student.Age < 30)  \  #where
#        .all()  #하나만 가져오고 싶으면 one()

result = session.query(Student).filter(Student.Age < 30).all()
for row in result:
    print type(row.Name), row.Age


<type 'str'> 24
<type 'str'> 25
<type 'str'> 25
<type 'str'> 27
<type 'str'> 22
<type 'str'> 20
[<__main__.Student object at 0x03E21150>, <__main__.Student object at 0x03BC6650>, <__main__.Student object at 0x03E21190>, <__main__.Student object at 0x03E211D0>, <__main__.Student object at 0x03E21210>, <__main__.Student object at 0x03E212B0>]


* insert / update

In [ ]:
# insert
s2 = Student(Name = 'Kevin', Age = 40, MajorCode = 'CS')
session.add(s2)
session.commit()

# update
# one vs first
# one은 한개만 찾아옴. 없거나 복수개 있다면 exception 발생
# first는 가장 상위에 있는 한개만 찾아옴. 없으면 None 반환
row = session.query(Student).filter(Student.Name == 'Kevin').one()
row.Name = 'Cindy'
session.commit()

 * delete

In [ ]:
session.query(Student).filter(Student.Name == 'Cindy').delete()
session.commit()

 * group by

In [ ]:
result = session.query(Student).group_by(Student.MajorCode).all()
for row in result:
    print row

* order by

In [ ]:
# 기본적으로 오름차순, 내림차순의 경우 desc 이용
from sqlalchemy import desc

session.query(Student).order_by(Student.Age).all()
for row in result:
    print row
    
session.query(Student).order_by(desc(Student.Age)).all()
for row in result:
    print row
    
session.query(Student).order_by(desc(Student.Age), MajorCode).all()
for row in result:
    print row

 * subquery

In [ ]:
stmt = session.query(func.avg(Score.Score)).subquery()
result = session.query(Score).group_by(Score.StudentID).having(func.avg(Score.Score) < stmt)
for row in result:
    print row.StudentId, row.Score

 * join

In [ ]:
'''select sc.StudentID, s.Name, avg(sc.Score) 
	from scores sc
    join students s
    on sc.StudentID = s.ID
    group by sc.StudentID;'''

from sqlalchemy import func
result = session.query(Score.StudentID, Student.Name, func.avg(Score.Score)) \
                .join(Student, Score.StudentID == Student.ID) \
                .group_by(Score.StudentID) \
                .all()
            
for row in result:
    print row
    
# 모든 필드를 가져옴
result = session.query(Score, Student) \
                .join(Student, Score.StudentID == Student.ID) \
                .group_by(Score.StudentID) \
                .all()
            
for row in result:
    print row.Student.ID, row.Student.Name

* 연습문제)
 0. python orm 으로 진행하시면 됩니다.
 1. 나이가 30 이상인 학생들의 평균성적은 얼마인가요?
 2. 나이가 27 이하인 학생들은 어떤 과목을 수강하나요?
 3. 과목별 평균 성적을 구하세요.


* 연습문제)
  1. world database로 선택해서 진행하시면 됩니다.
  2. GNP가 가장 높은 나라는?
  3. 각 나라의 주요 도시별 평균 인구수는?
  4. 기대수명이 평균보다 낮은 나라들의 평균 GNP는?
  

* API
 - HTTP request를 받아 document(HTML, json, xml 등의)를 response로 반환
 - python의 경우, Django, Flask, Tornado 등의 module 이용 가능
 - 거의 99% 이상의 api가 DB에서 값을 가져오거나 DB에 값을 쓰게 동작 함
 
* REST API
 - Representational state transfer 
 - 웹에 존재하는 모든 리소스를 URL과 매핑하고, CRUD 동작과 HTTP method를 연결하여 사용

In [6]:
from flask import Flask, jsonify
from models import Student
from orm_test2 import Session

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/test')
def hello_json():
    data = {'name' : 'Aaron', 'family' : 'Byun'}
    return jsonify(data)

@app.route('/users/<user_name>')
def get_user(user_name):
    session = Session()
    result = session.query(Student).filter(Student.Name == user_name).all()
    
    data = []
    for row in result:
        data.append({'name' : row.Name, 'age' : row.Age, 'major_code' : row.MajorCode})
        
    return jsonify(data)
    

if __name__ == '__main__':
    app.run(debug = True, host = '0.0.0.0', port = 5000)

ImportError: No module named flask

* 실습 및 과제)
 - 배운 것으로 종합적 설계를 진행해보아요.
   + 뉴스 크롤러 및 뉴스 검색 시스템 설계
   + 요구사항
     1. 매일 특정한 시각에 그날의 네이버 IT 뉴스(타이틀, 내용)를 모두 긁어오는 모듈이 필요
     2. 크롤러는 데이터를 rdb에 저장 (타이틀, 내용, 저장시각)
     3. 사용자는 API를 이용하여 키워드로 뉴스 검색이 가능
   + 해야할일
     1. class 설계 및 구현
     2. table scheme 설계 및 생성
     3. flask web API 구현
  
 

* 실습 및 과제를 위해 필요한 것들
 - cron, tmux, vi

  - crontab 사용법
    + cron 파일 위치 (/etc/crontab)
      - m h dom mon dow user command (분, 시각, 일, 월, 요일, 사용자, 명령어)
      - 30 09 10 06 * ubuntu python test.py       6월 10일 9시 30분에 실행
      - 00 11,16 * * * ubuntu python test.py      매일 11시, 16시에 실행
      - 00 09-18 * * 1-5 ubuntu python test.py    월-금 9시, 10시...18시에 실행
      - 00/10 * * * *    ubuntu python test.py     10분마다 실행
      
    
  - tmux 사용법
    + tmux new -s [session]  새로운 세션 생성
    + tmux attach -t [session] 세션에 접속
    + tmux ls 현재 실행중인 세션 나열
    
    + ctrl + b, c - 세션 새 윈도우 생성
    + ctrl + b, w - 세션 윈도우 리스트로 이동
    + ctrl + b, , - 윈도우 이름 변경
    + ctrl + b, [ - 윈도우 스크롤링
    + ctrl + b, & - 윈도우 삭제
    
    * tmux script 작성
      + tmux send -t [session]:[no] "command" ENTER
      + e.g)
       + tmux send -t dev:7 "python /home/dev/test.py" ENTER
       
       
  - vi에서 한글 작성 중 한글이 깨져보일 시
    - sudo apt-get install language-pack-ko
    - sudo locale-gen ko_KR.UTF-8
    - sudo dpkg-reconfigure locales
    + vi ~/.vimrc 에서 아래 두 줄 추가  
      - set encoding=utf-8
      - set fileencodings=utf-8,euckr